# The shortest chain which contains all hexamers from all 5 bases (A,T,C,G,U).

## How to generate the reads?
Create a slots (`np.array`) for reads.\
If the given hexamer does not exists in any reads, then append it somewhere with the following rules:
- Append the hexamer to the read where the overlap is the biggest. 
    - If more than one exists append the shortest read.
- If there is no overlap anywhere, then 
    - append the hexamert to a new read (if it is possible) OR
    - append to the shorthest existing read

### Todo
- how can the `Too many frequently occurring hexamers.` problem solved?

## Todo later
- check the G-C frequency for all reads

In [1]:
# packages
import numpy as np
import itertools

In [2]:
# functions
def combine_strings(string):
    ''' Add strings to each other. '''
    return ''.join(string)


def generate_k_mer_list(possible_bases, k_mer):
    ''' All possible repeated variations of k-mers. '''
    itr_prod = itertools.product(possible_bases, repeat=k_mer)
    k_mer_list = []
    for i in range(len(possible_bases)**k_mer):
        k_mer_list.append(combine_strings(next(itr_prod)))
    return np.array(k_mer_list)


def shortest_chain(k_mers): ## --> ChatGPT code, promt: given the following array; create the shortest string which contains all element of the given array
    '''
    With the help of ChatGPT, the shortest string that contains all possible hexamers,
    extending it only as much as necessary so that if a given k-mer already exists in the chain, it is not added again.
    '''
    result = k_mers[0]
    for i in range(1, len(k_mers)):
        overlap = len(k_mers[i]) - 1
        # If the k-mer already exists in the word chain, do not append it to the result.
        if result.count(k_mers[i]) == 0: # added to chatGPT code
            while overlap >= 0: # Backtracking until the k-mer matches the end of the chain.
                if result.endswith(k_mers[i][:overlap]):
                    break
                overlap -= 1
            result += k_mers[i][overlap:]
    return result


def kmers_sorted_by_frequency(chains, k_mers):
    ''' The frequency of possible k-mers in a given sequence in descending order. '''
    numbers = {}
    for element in k_mers:
        n=0
        for chain in chains:
            n += chain.count(element)
            numbers[element] = n
    sort_numbers = np.array(sorted(numbers.items(), key=lambda x:x[1], reverse=True))
    return sort_numbers


def run_chech(chains, k_mers, print_count=True, num_of_skipped_bases=0):
    ''' Counting identical frequencies --> how many k-mers appear n times in a sequence. '''
    sort_numbers = kmers_sorted_by_frequency(chains, k_mers)
    count = []
    for i in range(1, int(sort_numbers[0,1])+1):
        count.append([i, sum(np.array(sort_numbers[:,1], dtype=int) == i)])
    # missing elemets 
    count.append([0, sum(np.array(sort_numbers[:,1], dtype=int) == 0)])
    if print_count:
        print('\\begin{run_chech()}')
        print(f'  Length of the chain: {len(combine_strings(chains)) + num_of_skipped_bases}')
        print('  frequency - the number of k-mers:')
        for i in range(len(count)):
            # print if element type is not missing
            if count[i][1] !=0:
                print(f'  {count[i][0]} - {count[i][1]}')
        print('\end{run_chech()}')
    else:
        return count  
    return None


def count_bases(shortest_chain, print_count=True):
    ''' Count and print the number of bases. '''
    bases = ['A', 'T', 'C', 'G', 'U']
    count = {b: shortest_chain.count(b) for b in bases}
    # Order the dictionary by count in descending order
    count = dict(sorted(count.items(), key=lambda item: item[1], reverse=True))
    if print_count:
        print('\\begin{count_bases()}')
        for base, cnt in count.items():
            print(f'  {base}: {cnt}')
        print('\end{count_bases()}')
        return None
    return count


def calculate_GC_ratio(reads, tolerance=0.03, print_out_of_tolerance=False):
    ''' Calculate the GC ration. '''
    G_ratios, C_ratios = [], []
    out_of_tolerance = []
    for read in reads:
        num_of_G = read.count('G')
        num_of_C = read.count('C')
        sum_GC = num_of_G + num_of_C
        G_ratio = num_of_G/sum_GC
        G_ratios.append(G_ratio) 
        C_ratios.append(1 - G_ratio) 
        if G_ratio >= 0.5+tolerance or G_ratio <= 0.5-tolerance:
            out_of_tolerance.append(True)
        else:
            out_of_tolerance.append(False)
    if print_out_of_tolerance:
        print(f'{sum(out_of_tolerance)}/{len(reads)} (={(sum(out_of_tolerance)/len(reads)):.2f}) are out of tolerance.')
    return G_ratios, C_ratios, out_of_tolerance 

In [3]:
hexamers = generate_k_mer_list('ATCGU', 6)

print(f'Length without overlap: {len(combine_strings(hexamers))}')
shortest_hexamer_chain = shortest_chain(hexamers)
print(f'Length of the shortest string: {len(shortest_hexamer_chain)}')
print(f'It became {round(100 - len(shortest_hexamer_chain)/len(combine_strings(hexamers))*100, 3)}% shorter compared to the concatenated chain.')

Length without overlap: 93750
Length of the shortest string: 15635
It became 83.323% shorter compared to the concatenated chain.


---
---
---

Calculate the minimum number of slots.
- Each slot must have 240 bases or fewer.
- The maximum length of the combined chain must be less than 20,000 bases.

$\xrightarrow{} 20000/240\simeq 83$ **or more** slots needed

In [4]:
def generate_reads_v2(k_mers, num_of_reads, sorted_k_mers, print_rand_choiches=False):
    # random seed for reproducible result
    np.random.seed(137)

    fill_next_read = 1
    counter_for_random_choices = 0
    # the result will be stored in 'reads' var. as a numpy array
    reads = np.empty(num_of_reads, dtype='U300')
    # fill only the first read (originl case)
    if sorted_k_mers:
        reads[0] = k_mers[0]
        start_for_loop = 1
    # or fill all read with an element  (shuffled case)
    else:  
        for i in range(num_of_reads):
            reads[i] = k_mers[i]
        start_for_loop = num_of_reads

    for i in range(start_for_loop, len(k_mers)):
            
        # if the k-mer does not exists in any read, then put append it using the rules.
        if not any(k_mers[i] in read for read in reads):
                        
            overlaps_in_reads = []
            # Iterate trough all reads
            for j in range(len(reads)):   
                # the number of possible overlapping bases
                overlap = len(k_mers[0]) - 1
                # backtracking until the k-mer matches to the end of read j
                while overlap >= 0: 
                        # if the read end with the same character(s) as the hexamer begin.
                        if reads[j].endswith(k_mers[i][:overlap]):
                            # save the number of overlapping characters
                            overlaps_in_reads.append(overlap)
                            # break the while loop
                            break
                        # there is no overlap with n character --> try with n-1
                        overlap -= 1
        
            # if no overlap --> all elements are 0 in 'overlaps_in_reads'
            if np.all(np.array(overlaps_in_reads) == 0):
                # fill a new slot until there are no unfilled ones remaining
                if fill_next_read < num_of_reads:
                        reads[fill_next_read]+=k_mers[i]
                        fill_next_read+=1
                # else find the shortest read, and append the hexamer
                else:
                        # get the length of all read
                        length_of_all_read = np.vectorize(len)(reads)
                        # find all indices of the reads that are the shortest
                        min_read_len_idx = np.where(length_of_all_read == np.min(length_of_all_read))[0]
                        # randomly append the k-mer to the shortest read
                        reads[np.random.choice(min_read_len_idx)] += k_mers[i]
                        # add one to 'counter_for_random_choices' if it is needed
                        if len(min_read_len_idx) > 1: counter_for_random_choices +=1
                
            # else there is/are overlap(s) somewhere
            else:          
                # find the indices of maximal overlaps
                max_overlaps_indices = np.where(overlaps_in_reads == np.max(overlaps_in_reads))[0]
                # if more than one minimal overlap exists
                if len(max_overlaps_indices) > 1:
                        # get the length of selected reads
                        length_of_selected_reads = np.vectorize(len)(reads[max_overlaps_indices])
                        #too_long_reads = np.where(np.vectorize(len)(reads) > 235)[0]
                        #length_of_selected_reads = np.setdiff1d(length_of_selected_reads, too_long_reads)
                        # find the indices of the minimal read lenght
                        min_read_len_idx = np.where(length_of_selected_reads == np.min(length_of_selected_reads))[0]
                        # append the hexamer (w/o overlap) to the shortest read
                        reads[max_overlaps_indices[np.random.choice(min_read_len_idx)]] += k_mers[i][max(overlaps_in_reads):]
                        if len(min_read_len_idx) > 1: counter_for_random_choices +=1 
                # else one minimal overlap exists
                else:
                        #append the hexamer (w/o overlap)
                        reads[max_overlaps_indices[0]] += k_mers[i][max(overlaps_in_reads):]
                
    if print_rand_choiches:
        print('number of random choices:', counter_for_random_choices)
    return reads

In [5]:
reads = generate_reads_v2(hexamers, num_of_reads=115, sorted_k_mers=True)

In [6]:
np.vectorize(len)(reads)

array([227, 237, 227, 227, 229, 227, 235, 228, 232, 229, 240, 225, 225,
       227, 231, 234, 226, 230, 226, 227, 228, 227, 231, 225, 226, 229,
       230, 230, 233, 225, 228, 235, 228, 226, 230, 227, 229, 230, 228,
       229, 230, 234, 229, 228, 231, 226, 228, 228, 230, 229, 228, 228,
       230, 235, 227, 230, 227, 225, 239, 230, 227, 230, 228, 226, 230,
       228, 229, 227, 225, 228, 228, 230, 229, 228, 227, 227, 228, 225,
       225, 228, 235, 229, 236, 227, 227, 228, 225, 230, 230, 228, 240,
       230, 227, 226, 229, 228, 228, 233, 242, 229, 228, 230, 227, 226,
       231, 230, 236, 225, 228, 230, 230, 226, 229, 226, 230])

In [7]:
max(np.vectorize(len)(reads))

242

In [8]:
run_chech(reads, hexamers)

\begin{run_chech()}
  Length of the chain: 26347
  frequency - the number of k-mers:
  1 - 7973
  2 - 5548
  3 - 1802
  4 - 263
  5 - 35
  6 - 3
  7 - 1
\end{run_chech()}


In [9]:
G_ratios, C_ratios, out_of_tolerance = calculate_GC_ratio(reads, print_out_of_tolerance=True)

53/115 (=0.46) are out of tolerance.


### --> Still too many frequently occurring hexamers.
### --> And more longer chain lenght.
The problem of ratio is unsolved here also.

---
# With  random shuffle: (v4.2)

In [10]:
hexamers2 = hexamers.copy()
np.random.seed(137)
np.random.shuffle(hexamers2)
reads2 = generate_reads_v2(hexamers2, num_of_reads=110, sorted_k_mers=False)

In [11]:
max(np.vectorize(len)(reads2))

244

In [12]:
np.vectorize(len)(reads2)

array([232, 226, 224, 228, 226, 227, 231, 239, 228, 235, 232, 230, 226,
       234, 232, 227, 235, 231, 225, 232, 235, 234, 231, 232, 238, 223,
       223, 231, 225, 227, 223, 231, 229, 222, 229, 229, 228, 233, 229,
       226, 231, 228, 225, 217, 231, 225, 224, 230, 224, 228, 223, 225,
       228, 226, 223, 240, 229, 230, 233, 229, 232, 227, 224, 233, 232,
       234, 224, 235, 227, 235, 228, 228, 232, 230, 233, 229, 227, 224,
       226, 232, 244, 234, 236, 238, 228, 221, 223, 225, 229, 231, 225,
       229, 224, 228, 226, 233, 230, 228, 222, 226, 229, 225, 226, 232,
       227, 231, 232, 236, 228, 234])

In [13]:
run_chech(reads2, hexamers2)

\begin{run_chech()}
  Length of the chain: 25199
  frequency - the number of k-mers:
  1 - 8616
  2 - 5272
  3 - 1494
  4 - 213
  5 - 30
\end{run_chech()}


In [14]:
G_ratios2, C_ratios2, out_of_tolerance2 = calculate_GC_ratio(reads2, print_out_of_tolerance=True)

62/110 (=0.56) are out of tolerance.
